<a href="https://colab.research.google.com/github/jeongminia/NLP_paper_study/blob/main/%08code/RAG_1110.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ai-application-with-llm

In [1]:
!pip install datasets llama-index==0.10.34 langchain-openai==0.1.6 "nemoguardrails[openai]==0.8.0" openai==1.25.1 chromadb==0.5.0 wandb==0.16.6 -qqq
!pip install llama-index-callbacks-wandb==0.1.2 -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

## 9.1절 검색 증강 생성(RAG)

### 예제 9.1. 데이터셋 다운로드 및 API 키 설정

In [27]:
import os
from datasets import load_dataset

os.environ["OPENAI_API_KEY"] = "" # api 키 입력

dataset = load_dataset('klue', 'mrc', split='train')
dataset[0]

{'title': '제주도 장마 시작 … 중부는 이달 말부터',
 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.',
 'news_category': '종합',
 'source': 'hankyung',
 'guid': 'klue-mrc-v1_train_12759',
 'is_impossible': False,
 'question_type': 1,
 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?',
 'answers': {'answer_start': [478, 478]

### 예제 9.2. 입베딩 벡터 변환
실습 데이터 중 첫 100개를 뽑아 임베딩 벡터로 변환하고 저장

In [33]:
from llama_index.core import Document, VectorStoreIndex

text_list = dataset[:100]['context']
documents = [Document(text=t.encode('ascii', 'ignore').decode('ascii')) for t in text_list]

# 벡터 데이터 베이스에 저장
index = VectorStoreIndex.from_documents(documents)  # VectorStoreIndex는 텍스트 데이터를 임베딩 벡터로 변환하는데, 기본적으로 OpenAI의 text-embedding-ada-002 모델을 사용

UnicodeEncodeError: 'ascii' codec can't encode characters in position 7-9: ordinal not in range(128)

❗❗ ascii 인코딩에서 처리할 수 없는 비ASCII 문자(한글 등)가 포함되어 발생
- 기본적으로 텍스트를 ASCII로 인코딩하려고 시도

### 예제 9.3 100개의 기사 본문 데이터에서 질문과 가까운 기사 찾기

In [ ]:
print(dataset[0]['question']) # 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?

# 검색 엔진
retrieval_engine = index.as_retriever(similarity_top_k=5, # 검색 결과에서 유사도가 가장 높은 5개의 기사를 반환
                                      verbose=True)
response = retrieval_engine.retrieve(
    dataset[0]['question']
)
print(len(response)) # 출력 결과: 5
print(response[0].node.text)

- `as_retriever` : index 객체에 저장된 벡터 데이터베이스를  메소드를 사용하여 검색 엔진으로 변환
- `retrieve` : 특정 질문(예: '북태평양 기단과…')을 입력하여, 질문과 유사한 기사 찾기

### 예제 9.4 라마인덱스를 활용해 검색 증강 생성 수행하기

In [ ]:
query_engine = index.as_query_engine(similarity_top_k=1) # 검색 엔진을 생성하는 또 다른 방식
response = query_engine.query(
    dataset[0]['question']
)

print(response)
# 장마전선에서 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 한 달 정도입니다.

| 예제 번호 | 메소드            | 반환 결과 수 | 용도                               |
|-----------|-------------------|--------------|------------------------------------|
| 예제 9.3  | `as_retriever()`  | 여러 개      | 질문과 유사한 여러 문서가 필요할 때 |
| 예제 9.4  | `as_query_engine()` | 한 개        | 질문에 대한 최적의 하나의 답변이 필요할 때 |

### 예제 9.5 라마인덱스 내부에서 검색 증강 생성을 수행하는 과정
코드 출처: https://docs.llamaindex.ai/en/stable/understanding/querying/querying.html

- RAG 방식을 사용하여, 질문에 대해 가장 유사한 문서를 찾고 그 내용을 기반으로 답변을 생성
- RetrieverQueryEngine을 통해 검색 및 응답 생성을 통합하여 간편하게 질문에 대한 답변을 생성

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# 검색을 위한 Retriever 생성
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=1,
)

# 검색 결과를 질문과 결합하는 synthesizer
response_synthesizer = get_response_synthesizer() # 검색된 결과를 기반으로 최종 응답을 생성

# 위의 두 요소를 결합해 쿼리 엔진 생성
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.7)], # 이 값보다 유사도가 낮은 문서는 제외
)

# RAG 수행
response = query_engine.query("북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?")
print(response)
# 장마전선에서 북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은 한 달 가량입니다.